In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.autograd import Variable
from PIL import Image

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

import pickle
import json

from models.questioner import QuestionerNet
import data
from vocab import VocabTagger
from resnet_feature_extractor import ResnetFeatureExtractor

In [ ]:
resnet_feature_extractor = ResnetFeatureExtractor()
vocab_tagger = VocabTagger()

In [ ]:
split = 'valid'

with open(data.get_gw_file(split), 'r') as f:
        for line in f:
            example = json.loads(line)
            img_path = data.get_coco_file(example['image']['file_name'])
            img = Image.open(img_path)
            if img.mode != 'RGB':
                img = img.convert('RGB')

plt.imshow(img)

In [ ]:
feature = torch.from_numpy(resnet_feature_extractor.get_image_features(img))
feature.unsqueeze_(0)
feature_var = Variable(feature.cuda(), volatile=True)

In [ ]:
questioner_net = QuestionerNet().cuda()
utterance, h = questioner_net.sample(feature_var)
print(vocab_tagger.get_question_tokens(utterance))